In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ACTION_SPACE = ('U', 'D', 'L', 'R')

class windyGridWorld:
    
    def __init__(self,rows,cols,startPosition):
        self.rows = rows
        self.cols = cols
        self.i = startPosition[0]
        self.j = startPosition[1]
        
    
    def setOnGrid(self,rewards,actions,probs):
        self.rewards = rewards
        self.actions = actions
        self.probs = probs
    
    def setState(self,s):
        self.i = s[0]
        self.j = s[1]
    
    def move(self,action):
        s = (self.i,self.j)
        a = action
        next_state_probs = self.probs[(s,a)]
        next_states = next_state_probs.keys()
        next_probs = next_state_probs.values()
        next_state_idx = np.random.choice(len(next_states),p = next_probs)
        next_state = next_states[next_state_idx]
        self.i,self.j = next_state
        return self.rewards.get(next_state,0)
                
    def allStates(self):
        return set(self.actions.keys()) | set(self.rewards.keys())
        
    def allActions(self):
        return np.array(['U','D','L','R'])
    
    def currentState(self):
        return (self.i,self.j)
    
    def isTerminal(self,s):
        return s not in self.actions
    
    def reset(self):
        self.i = 2
        self.j = 0
        return (self.i,self.j)
    
    def gameOver(self):
        if (self.i==self.maxX and self.j==self.maxY):
            return True
        else:
            return False
            
    
def windyGrid():
    g = windyGridWorld(2+1,3+1,(2,0))
    rewards = {(0,3): +1 ,(1,3): -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
    }
    
    probs = {
        ((2, 0), 'U'): {(1, 0): 1.0},
        ((2, 0), 'D'): {(2, 0): 1.0},
        ((2, 0), 'L'): {(2, 0): 1.0},
        ((2, 0), 'R'): {(2, 1): 1.0},
        ((1, 0), 'U'): {(0, 0): 1.0},
        ((1, 0), 'D'): {(2, 0): 1.0},
        ((1, 0), 'L'): {(1, 0): 1.0},
        ((1, 0), 'R'): {(1, 0): 1.0},
        ((0, 0), 'U'): {(0, 0): 1.0},
        ((0, 0), 'D'): {(1, 0): 1.0},
        ((0, 0), 'L'): {(0, 0): 1.0},
        ((0, 0), 'R'): {(0, 1): 1.0},
        ((0, 1), 'U'): {(0, 1): 1.0},
        ((0, 1), 'D'): {(0, 1): 1.0},
        ((0, 1), 'L'): {(0, 0): 1.0},
        ((0, 1), 'R'): {(0, 2): 1.0},
        ((0, 2), 'U'): {(0, 2): 1.0},
        ((0, 2), 'D'): {(1, 2): 1.0},
        ((0, 2), 'L'): {(0, 1): 1.0},
        ((0, 2), 'R'): {(0, 3): 1.0},
        ((2, 1), 'U'): {(2, 1): 1.0},
        ((2, 1), 'D'): {(2, 1): 1.0},
        ((2, 1), 'L'): {(2, 0): 1.0},
        ((2, 1), 'R'): {(2, 2): 1.0},
        ((2, 2), 'U'): {(1, 2): 1.0},
        ((2, 2), 'D'): {(2, 2): 1.0},
        ((2, 2), 'L'): {(2, 1): 1.0},
        ((2, 2), 'R'): {(2, 3): 1.0},
        ((2, 3), 'U'): {(1, 3): 1.0},
        ((2, 3), 'D'): {(2, 3): 1.0},
        ((2, 3), 'L'): {(2, 2): 1.0},
        ((2, 3), 'R'): {(2, 3): 1.0},
        ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
        ((1, 2), 'D'): {(2, 2): 1.0},
        ((1, 2), 'L'): {(1, 2): 1.0},
        ((1, 2), 'R'): {(1, 3): 1.0},
      }
    g.setOnGrid(rewards, actions, probs)
    return g



def windy_grid_penalized(step_cost=-0.1):
    g = windyGridWorld(3, 4, (2, 0))
    rewards = {
        (0, 0): step_cost,
        (0, 1): step_cost,
        (0, 2): step_cost,
        (1, 0): step_cost,
        (1, 2): step_cost,
        (2, 0): step_cost,
        (2, 1): step_cost,
        (2, 2): step_cost,
        (2, 3): step_cost,
        (0, 3): 1,
        (1, 3): -1
    }
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
      }

  # p(s' | s, a) represented as:
  # KEY: (s, a) --> VALUE: {s': p(s' | s, a)}
    probs = {
        ((2, 0), 'U'): {(1, 0): 1.0},
        ((2, 0), 'D'): {(2, 0): 1.0},
        ((2, 0), 'L'): {(2, 0): 1.0},
        ((2, 0), 'R'): {(2, 1): 1.0},
        ((1, 0), 'U'): {(0, 0): 1.0},
        ((1, 0), 'D'): {(2, 0): 1.0},
        ((1, 0), 'L'): {(1, 0): 1.0},
        ((1, 0), 'R'): {(1, 0): 1.0},
        ((0, 0), 'U'): {(0, 0): 1.0},
        ((0, 0), 'D'): {(1, 0): 1.0},
        ((0, 0), 'L'): {(0, 0): 1.0},
        ((0, 0), 'R'): {(0, 1): 1.0},
        ((0, 1), 'U'): {(0, 1): 1.0},
        ((0, 1), 'D'): {(0, 1): 1.0},
        ((0, 1), 'L'): {(0, 0): 1.0},
        ((0, 1), 'R'): {(0, 2): 1.0},
        ((0, 2), 'U'): {(0, 2): 1.0},
        ((0, 2), 'D'): {(1, 2): 1.0},
        ((0, 2), 'L'): {(0, 1): 1.0},
        ((0, 2), 'R'): {(0, 3): 1.0},
        ((2, 1), 'U'): {(2, 1): 1.0},
        ((2, 1), 'D'): {(2, 1): 1.0},
        ((2, 1), 'L'): {(2, 0): 1.0},
        ((2, 1), 'R'): {(2, 2): 1.0},
        ((2, 2), 'U'): {(1, 2): 1.0},
        ((2, 2), 'D'): {(2, 2): 1.0},
        ((2, 2), 'L'): {(2, 1): 1.0},
        ((2, 2), 'R'): {(2, 3): 1.0},
        ((2, 3), 'U'): {(1, 3): 1.0},
        ((2, 3), 'D'): {(2, 3): 1.0},
        ((2, 3), 'L'): {(2, 2): 1.0},
        ((2, 3), 'R'): {(2, 3): 1.0},
        ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
        ((1, 2), 'D'): {(2, 2): 1.0},
        ((1, 2), 'L'): {(1, 2): 1.0},
        ((1, 2), 'R'): {(1, 3): 1.0},
  }
    g.setOnGrid(rewards, actions, probs)
    return g


def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")


def print_policy(P, g):
      for i in range(g.rows):
            print("---------------------------")
            for j in range(g.cols):
                a = P.get((i,j), ' ')
                print("  %s  |" % a, end="")
            print("")



In [24]:
rewards = {}
transitionProbs = {}
grid = windyGrid()
#grid = windy_grid_penalized(-0.3)   #Compare these values results: stepcost= -0.1 , -0.3 , -2 , -10                  
for (s, a), v in grid.probs.items():
    for s2, p in v.items():
        transitionProbs[(s, a, s2)] = p
        rewards[(s, a, s2)] = grid.rewards.get(s2, 0)

#print("\n\n")
v = {}
for s in grid.allStates():
    v[s] = 0

    
gamma = 0.9
it = 0 
print_values(grid.rewards,grid)
print('\n\n')
#policy evaluation
while True:   
    delta = 0
    for s in grid.allStates():
        if not grid.isTerminal(s):
            vOld = v[s]
            vBest = float('-inf')
            for a in ACTION_SPACE:
                vNew = 0
                for sPrime in grid.allStates():
                    vNew += transitionProbs.get((s,a,sPrime),0) * (rewards.get((s,a,sPrime),0) + gamma*v[sPrime])
                if vNew>vBest:
                    vBest = vNew
            v[s] = vBest
            delta = max(delta , np.abs(vOld - v[s]))
        #print("iter: ", it ,"delta: ",delta)
        
    it += 1

    if delta < 1e-3:
        break

################################
policy = {}
for s in grid.actions.keys():
    vBest = float('-inf')

    for a in ACTION_SPACE:
        vNew = 0

        for sPrime in grid.allStates():
            vNew += transitionProbs.get((s,a,sPrime),0) * (rewards.get((s,a,sPrime),0) + gamma*v[sPrime])
        if vNew>vBest:
            vBest = vNew
            aBest = a
    policy[s] = aBest

print_values(v,grid)
print('\n\n')
print_policy(policy,grid)

---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|



---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.48| 0.00|
---------------------------
 0.66| 0.59| 0.53| 0.48|



---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  D  |     |
---------------------------
  U  |  L  |  L  |  L  |


In [23]:
# policy behine unique nist ama value behine unique hast
# dar value iteration har marhale yek loop dare
# bar khalafe policy iteration ke kole marahel ham to loop hast